# 🧪 Experiments Report

This report aggregates experiment runs (train/val metrics, hyperparameters, tags) and produces summaries & charts.

### How to use
1. Drop a `runs.csv` **or** `runs.json` into the same folder as this notebook.
   - Expected columns/fields (flexible): `id, timestamp, experiment, tag, lr, batch_size, epochs, val_loss, val_acc, sharpe, drawdown, notes`.
2. Run all cells. If no data file is found, the notebook fabricates a small dummy dataset so you can see the visuals.
3. Tweak the filters at the top to select subsets (e.g., a specific experiment or tag) and re-run.

In [ ]:
import os, json, math, datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

plt.rcParams["figure.figsize"] = (10, 4)
DATA_DIR = Path(".")
CSV_PATH = DATA_DIR / "runs.csv"
JSON_PATH = DATA_DIR / "runs.json"

# ---- Filters (edit here) ----
FILTER_EXPERIMENT = None      # e.g., "sentiment_v2" or None
FILTER_TAG = None             # e.g., "baseline" or None
SORT_BY = "val_loss"          # default sort metric
TOP_K = 10

def _dummy(n=30, seed=7):
    rng = np.random.default_rng(seed)
    rows = []
    base = datetime.datetime.now()
    for i in range(n):
        ts = base - datetime.timedelta(hours=(n - i))
        lr = 10**rng.uniform(-5, -2)
        bs = int(rng.choice([16, 32, 64, 128]))
        epochs = int(rng.integers(5, 31))
        val_loss = float(rng.normal(0.35, 0.05)) + 0.1*(0.0005/lr) + 0.01*(64/bs)
        val_acc = float(np.clip(rng.normal(0.80, 0.05), 0.5, 0.99)) - 0.2*(val_loss-0.3)
        sharpe = float(np.clip(rng.normal(1.2, 0.4), -0.2, 3.0)) + 0.5*(0.85 - val_loss)
        drawdown = float(np.clip(rng.normal(0.10, 0.05), 0.02, 0.35)) + 0.2*(val_loss-0.3)
        rows.append({
            "id": f"run_{i:03d}",
            "timestamp": ts.isoformat(),
            "experiment": rng.choice(["sentiment_v2","regime_map","vol_surface","alpha_blend"]),
            "tag": rng.choice(["baseline","tuned","ablation","debug"]),
            "lr": lr,
            "batch_size": bs,
            "epochs": epochs,
            "val_loss": max(val_loss, 0.05),
            "val_acc": float(np.clip(val_acc, 0.5, 0.999)),
            "sharpe": sharpe,
            "drawdown": drawdown,
            "notes": "auto-dummy"
        })
    return pd.DataFrame(rows)

def _load_runs():
    if CSV_PATH.exists():
        df = pd.read_csv(CSV_PATH)
    elif JSON_PATH.exists():
        with open(JSON_PATH, "r") as f:
            data = json.load(f)
        df = pd.DataFrame(data)
    else:
        df = _dummy()
    # ensure expected columns exist
    for col in ["id","timestamp","experiment","tag","val_loss","val_acc","sharpe","drawdown"]:
        if col not in df.columns:
            df[col] = np.nan
    # coerce types
    if "timestamp" in df.columns:
        df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    return df

df = _load_runs()
orig_rows = len(df)
if FILTER_EXPERIMENT:
    df = df[df["experiment"] == FILTER_EXPERIMENT]
if FILTER_TAG:
    df = df[df["tag"] == FILTER_TAG]
df = df.sort_values(by=[SORT_BY], ascending=True if SORT_BY.lower().endswith("loss") else False) # type: ignore
print(f"Loaded {orig_rows} runs; after filters: {len(df)}")
df.head(10)

## Summary Stats

In [ ]:
summary = {
    "n_runs": int(len(df)),
    "best_val_loss": float(df["val_loss"].min(skipna=True)),
    "best_val_acc": float(df["val_acc"].max(skipna=True)),
    "best_sharpe": float(df["sharpe"].max(skipna=True)),
    "min_drawdown": float(df["drawdown"].min(skipna=True)),
}
summary

## Top-K Runs

In [ ]:
cols = [c for c in ["id","timestamp","experiment","tag","val_loss","val_acc","sharpe","drawdown","lr","batch_size","epochs","notes"] if c in df.columns]
topk = df[cols].head(TOP_K).reset_index(drop=True)
topk

## Charts

- **Metric over time** (val_loss, val_acc)
- **Hyperparameter sweeps** (lr vs metrics; batch_size vs metrics)
- **Pareto frontier** (minimize val_loss, minimize drawdown / maximize sharpe)


In [ ]:
def _plot_series(x, y, title, xlabel, ylabel):
    plt.figure()
    plt.plot(x, y, marker='o', linewidth=1)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.show()

if "timestamp" in df.columns and df["timestamp"].notna().any():
    _plot_series(df["timestamp"], df["val_loss"], "Validation Loss over Time", "Time", "val_loss")
    if "val_acc" in df.columns:
        _plot_series(df["timestamp"], df["val_acc"], "Validation Accuracy over Time", "Time", "val_acc")

if "lr" in df.columns:
    _plot_series(np.log10(df["lr"].astype(float).replace(0,np.nan)), df["val_loss"], "val_loss vs log10(lr)", "log10(lr)", "val_loss")

if "batch_size" in df.columns:
    _plot_series(df["batch_size"], df["val_loss"], "val_loss vs batch_size", "batch_size", "val_loss")

# Pareto: minimize val_loss and drawdown, maximize sharpe (we'll use -sharpe to plot a 2D frontier)
if set(["val_loss","drawdown"]).issubset(df.columns):
    x = df["val_loss"].astype(float).values
    y = df["drawdown"].astype(float).values
    idx = np.argsort(x) # type: ignore
    x2, y2 = x[idx], y[idx]
    # lower envelope
    frontier_x, frontier_y = [], []
    cur = float("inf")
    for xv, yv in zip(x2, y2):
        if yv < cur:
            frontier_x.append(xv); frontier_y.append(yv); cur = yv
    plt.figure()
    plt.scatter(x, y, s=20) # type: ignore
    plt.plot(frontier_x, frontier_y, linewidth=2)
    plt.title("Pareto Frontier: val_loss vs drawdown (lower-left is better)")
    plt.xlabel("val_loss")
    plt.ylabel("drawdown")
    plt.grid(True)
    plt.show()


## Export: Markdown Summary
Creates a `EXPERIMENTS_SUMMARY.md` you can paste into PRs or wikis.

In [ ]:
lines = [
    "# Experiments Summary\n",
    f"Generated: {datetime.datetime.now().isoformat()}\n\n",
    f"Total runs: {summary['n_runs']}\n\n",
    f"- Best val_loss: {summary['best_val_loss']:.4f}\n",
    f"- Best val_acc:  {summary['best_val_acc']:.4f}\n",
    f"- Best sharpe:   {summary['best_sharpe']:.4f}\n",
    f"- Min drawdown:  {summary['min_drawdown']:.4f}\n\n",
    "## Top Runs\n",
]
for i, row in topk.iterrows():
    rid = row.get("id", f"#{i}")
    ts = row.get("timestamp", "-")
    exp = row.get("experiment", "-")
    tag = row.get("tag", "-")
    vl = row.get("val_loss", float("nan"))
    va = row.get("val_acc", float("nan"))
    sh = row.get("sharpe", float("nan"))
    dd = row.get("drawdown", float("nan"))
    lines.append(f"- **{rid}** | {ts} | {exp} / {tag} | val_loss={vl:.4f} | val_acc={va:.4f} | sharpe={sh:.2f} | dd={dd:.2f}\n")

with open("EXPERIMENTS_SUMMARY.md", "w") as f:
    f.writelines(lines)
print("Wrote EXPERIMENTS_SUMMARY.md")